# agg all the data versions here.

In [ ]:
import numpy as np
import pandas as pd
import sqlite3

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [ ]:
conn = sqlite3.connect('data/NBA-Boxscore-Database.sqlite')

In [ ]:
query = 'SELECT * from game_info'
game_info = pd.read_sql(query, con=conn)
game_info

In [ ]:
query = 'SELECT * from team_stats'
team_stats = pd.read_sql(query, con=conn)
team_stats

In [ ]:
query = 'SELECT * from player_stats'
player_stats = pd.read_sql(query, con=conn)
player_stats

In [ ]:
game_ids = list(game_info['game_id'].unique())

## Four Factor Team Aggregation

In [ ]:
def create_gid_query(game_id, date, lim, team='away'):
    away_gid_query = f'''
    SELECT gi.game_id
    FROM game_info gi
    WHERE (gi.away_team = (SELECT away_team FROM game_info WHERE game_id = '{game_id}')
            AND gi.date < '{date}')
       OR (gi.home_team = (SELECT away_team FROM game_info WHERE game_id = '{game_id}')
            AND gi.date < '{date}')
    ORDER BY gi.date DESC
    LIMIT {lim};
    '''
    
    home_gid_query = f'''
    SELECT gi.game_id
    FROM game_info gi
    WHERE (gi.away_team = (SELECT home_team FROM game_info WHERE game_id = '{game_id}')
            AND gi.date < '{date}')
       OR (gi.home_team = (SELECT home_team FROM game_info WHERE game_id = '{game_id}')
            AND gi.date < '{date}')
    ORDER BY gi.date DESC
    LIMIT {lim};
    '''
    
    if team == 'away':
        return away_gid_query
    elif team == 'home':
        return home_gid_query

In [ ]:
def create_4F_df(query, conn, team_stats, team='away'):
    
    gid_query_df = pd.read_sql(query, con=conn)
    stat_df = gid_query_df.merge(team_stats[['game_id', 'team', 'FG', 'FGA', 'FT', 'FTA', '3P', 'ORBp', 'TOV']])
    
    if team == 'away':
        stat_df = stat_df[stat_df['team'] == away_team].reset_index(drop=True)
    elif team =='home':
        stat_df = stat_df[stat_df['team'] == home_team].reset_index(drop=True)
    
    eFGp = (stat_df['FG'].mean() + (0.5 * stat_df['3P'].mean())) / stat_df['FGA'].mean()
    FTr = stat_df['FT'].mean() / stat_df['FGA'].mean()
    ORBp = stat_df['ORBp'].mean()*0.01
    TOVp = stat_df['TOV'].mean() / (stat_df['FGA'].mean() + (0.44*stat_df['FTA'].mean()) + stat_df['TOV'].mean())
    
    if team == 'away':
        comp_df = pd.DataFrame(data = [[eFGp, FTr, ORBp, TOVp]], columns=['a_eFGp', 'a_FTr', 'a_ORBp', 'a_TOVp'])
    elif team =='home':
        comp_df = pd.DataFrame(data = [[eFGp, FTr, ORBp, TOVp]], columns=['h_eFGp', 'h_FTr', 'h_ORBp', 'h_TOVp'])
    
    return comp_df

In [ ]:
four_factor_columns = list(game_info.columns) + ['a_eFGp', 'a_FTr', 'a_ORBp', 'a_TOVp', 'h_eFGp', 'h_FTr', 'h_ORBp', 'h_TOVp']
team_factor_10_df = pd.DataFrame(columns=four_factor_columns)

for game_id in game_ids:

    date = game_info[game_info['game_id'] == game_id]['date'].values[0]
    away_team = game_info[game_info['game_id'] == game_id]['away_team'].values[0]
    home_team = game_info[game_info['game_id'] == game_id]['home_team'].values[0]

    away_gid_query = create_gid_query(game_id = game_id, date=date, team='away')
    away_stats = create_4F_df(query=away_gid_query, conn=conn, team_stats=team_stats, team='away')

    home_gid_query = create_gid_query(game_id=game_id, date=date, team='home')
    home_stats = create_4F_df(query=home_gid_query, conn=conn, team_stats=team_stats, team='home')
    
    agg_stats = pd.concat([away_stats, home_stats], axis=1)
    gid_info = game_info[game_info['game_id'] == game_id].reset_index(drop=True)
    
    stats_4F = pd.concat([gid_info, agg_stats], axis=1)
    team_factor_10_df = four_factor_df.append(stats_4F, ignore_index=True)

In [ ]:
team_factor_10_df

In [ ]:
team_factor_10_df.to_csv('team_factor_10.csv')

In [ ]:
team_factor_20_df = pd.DataFrame(columns=four_factor_columns)

for game_id in game_ids:

    date = game_info[game_info['game_id'] == game_id]['date'].values[0]
    away_team = game_info[game_info['game_id'] == game_id]['away_team'].values[0]
    home_team = game_info[game_info['game_id'] == game_id]['home_team'].values[0]

    away_gid_query = create_gid_query(game_id = game_id, date=date, lim=20, team='away')
    away_stats = create_4F_df(query=away_gid_query, conn=conn, team_stats=team_stats, team='away')

    home_gid_query = create_gid_query(game_id=game_id, date=date, lim=20, team='home')
    home_stats = create_4F_df(query=home_gid_query, conn=conn, team_stats=team_stats, team='home')
    
    agg_stats = pd.concat([away_stats, home_stats], axis=1)
    gid_info = game_info[game_info['game_id'] == game_id].reset_index(drop=True)
    
    stats_4F = pd.concat([gid_info, agg_stats], axis=1)
    team_factor_20_df = four_factor_df.append(stats_4F, ignore_index=True)

In [ ]:
team_factor_20_df

In [ ]:
team_factor_20_df.to_csv('team_factor_20.csv')

In [ ]:
team_factor_30_df = pd.DataFrame(columns=four_factor_columns)

for game_id in game_ids:

    date = game_info[game_info['game_id'] == game_id]['date'].values[0]
    away_team = game_info[game_info['game_id'] == game_id]['away_team'].values[0]
    home_team = game_info[game_info['game_id'] == game_id]['home_team'].values[0]

    away_gid_query = create_gid_query(game_id = game_id, date=date, lim=30, team='away')
    away_stats = create_4F_df(query=away_gid_query, conn=conn, team_stats=team_stats, team='away')

    home_gid_query = create_gid_query(game_id=game_id, date=date, lim=30, team='home')
    home_stats = create_4F_df(query=home_gid_query, conn=conn, team_stats=team_stats, team='home')
    
    agg_stats = pd.concat([away_stats, home_stats], axis=1)
    gid_info = game_info[game_info['game_id'] == game_id].reset_index(drop=True)
    
    stats_4F = pd.concat([gid_info, agg_stats], axis=1)
    team_factor_30_df = four_factor_df.append(stats_4F, ignore_index=True)

In [ ]:
team_factor_30_df

In [ ]:
team_factor_30_df.to_csv('team_factor_30.csv')

## Player Four Factor Aggregation

In [ ]:
id_ref = player_stats[['player', 'team', 'game_id']]
four_factor_columns = ['game_id', 'a_eFGp', 'a_FTr', 'a_ORBp', 'a_TOVp', 'h_eFGp', 'h_FTr', 'h_ORBp', 'h_TOVp']

In [ ]:
player_factor_10 = pd.DataFrame(columns=four_factor_columns)

for game_id in game_ids:
    
    away_team = game_info['away_team'][game_info['game_id'] == game_id]
    home_team = game_info['home_team'][game_info['game_id'] == game_id]

    query = f'''select player, SUM(FG) as FG, SUM(FGA) as FGA, SUM(FT) as FT, SUM(FTA) as FTA, SUM("3P") as P3, SUM(ORBp) as ORBp, SUM(TOV) as TOV
                from ( select ROW_NUMBER() OVER (PARTITION BY d.player order by d.date desc ) r, d.*  
                    from ( select game_id, player from player_stats ps where ps.game_id={game_id} ) gs
                    inner join ( select b.date,  a.* from player_stats a inner join game_info b on a.game_id = b.game_id ) d
                        on gs.player=d.player
                ) k
                where (k.r BETWEEN 2 and 11)
                group by player;'''

    stat_df = pd.read_sql(query, con=conn)
    player_agg_stats = pd.merge(stat_df, id_ref[['player', 'team']][id_ref['game_id'] == game_id], on=['player'])
    team_avg_stats = player_agg_stats.groupby('team').sum()/10
    
    a_eFGp = float( ( team_avg_stats.loc[away_team]['FG'] + (0.5*team_avg_stats.loc[away_team]['P3']) ) / team_avg_stats.loc[away_team]['FGA'] )
    a_FTr = float( team_avg_stats.loc[away_team]['FT'] / team_avg_stats.loc[away_team]['FGA'] )
    a_ORBp = float( team_avg_stats.loc[away_team]['ORBp']*0.01 )
    a_TOVp = float( team_avg_stats.loc[away_team]['TOV'] / ( team_avg_stats.loc[away_team]['FGA'] + 0.44*team_avg_stats.loc[away_team]['FTA'] + team_avg_stats.loc[away_team]['TOV'] ) )
    
    h_eFGp = float( ( team_avg_stats.loc[home_team]['FG'] + (0.5*team_avg_stats.loc[home_team]['P3']) ) / team_avg_stats.loc[home_team]['FGA'] )
    h_FTr = float( team_avg_stats.loc[home_team]['FT'] / team_avg_stats.loc[home_team]['FGA'] )
    h_ORBp = float( team_avg_stats.loc[home_team]['ORBp']*0.01 )
    h_TOVp = float( team_avg_stats.loc[home_team]['TOV'] / ( team_avg_stats.loc[home_team]['FGA'] + 0.44*team_avg_stats.loc[home_team]['FTA'] + team_avg_stats.loc[home_team]['TOV'] ) )

    stats = [game_id, a_eFGp, a_FTr, a_ORBp, a_TOVp, h_eFGp, h_FTr, h_ORBp, h_TOVp]
    idx = player_factor_10.shape[0]
    player_factor_10.loc[idx] = stats

player_factor_10 = game_info.merge(player_factor_10, on='game_id')
player_factor_10.to_csv('player_factor_10.csv')

In [ ]:
player_factor_10.tail()

In [ ]:
player_factor_20 = pd.DataFrame(columns=four_factor_columns)

for game_id in game_ids:
    
    away_team = game_info['away_team'][game_info['game_id'] == game_id]
    home_team = game_info['home_team'][game_info['game_id'] == game_id]

    query = f'''select player, SUM(FG) as FG, SUM(FGA) as FGA, SUM(FT) as FT, SUM(FTA) as FTA, SUM("3P") as P3, SUM(ORBp) as ORBp, SUM(TOV) as TOV
                from ( select ROW_NUMBER() OVER (PARTITION BY d.player order by d.date desc ) r, d.*  
                    from ( select game_id, player from player_stats ps where ps.game_id={game_id} ) gs
                    inner join ( select b.date,  a.* from player_stats a inner join game_info b on a.game_id = b.game_id ) d
                        on gs.player=d.player
                ) k
                where (k.r BETWEEN 2 and 21)
                group by player;'''

    stat_df = pd.read_sql(query, con=conn)
    player_agg_stats = pd.merge(stat_df, id_ref[['player', 'team']][id_ref['game_id'] == game_id], on=['player'])
    team_avg_stats = player_agg_stats.groupby('team').sum()/20
    
    a_eFGp = float( ( team_avg_stats.loc[away_team]['FG'] + (0.5*team_avg_stats.loc[away_team]['P3']) ) / team_avg_stats.loc[away_team]['FGA'] )
    a_FTr = float( team_avg_stats.loc[away_team]['FT'] / team_avg_stats.loc[away_team]['FGA'] )
    a_ORBp = float( team_avg_stats.loc[away_team]['ORBp']*0.01 )
    a_TOVp = float( team_avg_stats.loc[away_team]['TOV'] / ( team_avg_stats.loc[away_team]['FGA'] + 0.44*team_avg_stats.loc[away_team]['FTA'] + team_avg_stats.loc[away_team]['TOV'] ) )
    
    h_eFGp = float( ( team_avg_stats.loc[home_team]['FG'] + (0.5*team_avg_stats.loc[home_team]['P3']) ) / team_avg_stats.loc[home_team]['FGA'] )
    h_FTr = float( team_avg_stats.loc[home_team]['FT'] / team_avg_stats.loc[home_team]['FGA'] )
    h_ORBp = float( team_avg_stats.loc[home_team]['ORBp']*0.01 )
    h_TOVp = float( team_avg_stats.loc[home_team]['TOV'] / ( team_avg_stats.loc[home_team]['FGA'] + 0.44*team_avg_stats.loc[home_team]['FTA'] + team_avg_stats.loc[home_team]['TOV'] ) )

    stats = [game_id, a_eFGp, a_FTr, a_ORBp, a_TOVp, h_eFGp, h_FTr, h_ORBp, h_TOVp]
    idx = player_factor_20.shape[0]
    player_factor_20.loc[idx] = stats

player_factor_20 = game_info.merge(player_factor_20, on='game_id')
player_factor_20.to_csv('player_factor_20.csv')

In [ ]:
player_factor_20.tail()

## Full Team Aggregation 

In [ ]:
def create_stats_df(query, conn, stats, team='away'):
    
    gid_query_df = pd.read_sql(query, con=conn)
    stat_df = gid_query_df.merge(stats)
    
    if team == 'away':
        stat_df = stat_df[stat_df['team'] == away_team].reset_index(drop=True)
        stat_agg = stat_df.loc[:,'FG':'TOVp'].mean()
        stat_agg = stat_agg.to_frame().T
        columns = [f'a_{col}' for col in stat_agg.columns]
        col_name_dict = dict(zip(stat_agg.columns, columns))
        stat_agg.rename(columns=col_name_dict, inplace=True)
        
    elif team =='home':
        stat_df = stat_df[stat_df['team'] == home_team].reset_index(drop=True)
        stat_agg = stat_df.loc[:,'FG':'TOVp'].mean()
        stat_agg = stat_agg.to_frame().T
        columns = [f'h_{col}' for col in stat_agg.columns]
        col_name_dict = dict(zip(stat_agg.columns, columns))
        stat_agg.rename(columns=col_name_dict, inplace=True)
    
    return stat_agg

In [ ]:
stats_df = team_stats.drop(['MP', 'PM', 'USGp', 'ORtg', 'DRtg', 'BPM'], axis=1)
final_df_columns = list(game_info.columns) + [f'a_{column}' for column in stats_df.loc[:,'FG':'TOVp']] + [f'h_{column}' for column in stats_df.loc[:,'FG':'TOVp']]

In [ ]:
team_full_10 = pd.DataFrame(columns=final_df_columns)

for game_id in game_ids:

    date = game_info[game_info['game_id'] == game_id]['date'].values[0]
    away_team = game_info[game_info['game_id'] == game_id]['away_team'].values[0]
    home_team = game_info[game_info['game_id'] == game_id]['home_team'].values[0]

    away_gid_query = create_gid_query(game_id = game_id, date=date, lim=10, team='away')
    away_stats = create_stats_df(query=away_gid_query, conn=conn, stats=stats_df, team='away')

    home_gid_query = create_gid_query(game_id=game_id, date=date, lim=10, team='home')
    home_stats = create_stats_df(query=home_gid_query, conn=conn, stats=stats_df, team='home')
    
    agg_stats = pd.concat([away_stats, home_stats], axis=1)
    gid_info = game_info[game_info['game_id'] == game_id].reset_index(drop=True)
    
    stats = pd.concat([gid_info, agg_stats], axis=1)
    team_full_10 = team_full_10.append(stats, ignore_index=True)

team_full_10.to_csv('team_full_10.csv')

In [ ]:
team_full_20 = pd.DataFrame(columns=final_df_columns)

for game_id in game_ids:

    date = game_info[game_info['game_id'] == game_id]['date'].values[0]
    away_team = game_info[game_info['game_id'] == game_id]['away_team'].values[0]
    home_team = game_info[game_info['game_id'] == game_id]['home_team'].values[0]

    away_gid_query = create_gid_query(game_id = game_id, date=date, lim=20, team='away')
    away_stats = create_stats_df(query=away_gid_query, conn=conn, stats=stats_df, team='away')

    home_gid_query = create_gid_query(game_id=game_id, date=date, lim=20, team='home')
    home_stats = create_stats_df(query=home_gid_query, conn=conn, stats=stats_df, team='home')
    
    agg_stats = pd.concat([away_stats, home_stats], axis=1)
    gid_info = game_info[game_info['game_id'] == game_id].reset_index(drop=True)
    
    stats = pd.concat([gid_info, agg_stats], axis=1)
    team_full_20 = team_full_20.append(stats, ignore_index=True)

team_full_20.to_csv('team_full_20.csv')

In [ ]:
team_full_30 = pd.DataFrame(columns=final_df_columns)

for game_id in game_ids:

    date = game_info[game_info['game_id'] == game_id]['date'].values[0]
    away_team = game_info[game_info['game_id'] == game_id]['away_team'].values[0]
    home_team = game_info[game_info['game_id'] == game_id]['home_team'].values[0]

    away_gid_query = create_gid_query(game_id = game_id, date=date, lim=30, team='away')
    away_stats = create_stats_df(query=away_gid_query, conn=conn, stats=stats_df, team='away')

    home_gid_query = create_gid_query(game_id=game_id, date=date, lim=30, team='home')
    home_stats = create_stats_df(query=home_gid_query, conn=conn, stats=stats_df, team='home')
    
    agg_stats = pd.concat([away_stats, home_stats], axis=1)
    gid_info = game_info[game_info['game_id'] == game_id].reset_index(drop=True)
    
    stats = pd.concat([gid_info, agg_stats], axis=1)
    team_full_30 = team_full_30.append(stats, ignore_index=True)

team_full_30.to_csv('team_full_30.csv')

In [ ]:
conn.close()